
# Preprocessing 
### This Notebook takes an image and the output is the binarized Image with horizontal staff lines
### It takes care of the enhancement of the image and choosing apropriate threshold


In [2]:
from commonfunctions import *
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.measure import find_contours
from skimage.draw import rectangle
from collections import Counter
import skimage.filters as fr
import skimage as sk
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [9]:
def binraization(img,n,t):

    outputimg = np.zeros(img.shape)
    intimg = np.zeros(img.shape)
    h = img.shape[1]
    w = img.shape[0]
    s= w//n
    for i in range(0,w):
        sum =0
        for j in range(0,h):
            sum = sum + img[i,j]
            if i ==0:
                intimg[i,j]= sum 
            else :
                intimg[i,j]= intimg[i-1,j] + sum 
    for i in range(0,w):
        for j in range(0,h):
            x1 = max(i - s//2,0)
            x2 = min(i + s//2,w-1)
            y1 = max(j - s//2,0) 
            y2 = min(j + s//2,h-1)
            count = (x2-x1)*(y2-y1)
            sum = intimg[x2,y2]-intimg[x2,y1-1]-intimg[x1-1,y2]+intimg[x1-1,y1-1]
            if(img[i,j]*count)<=(sum*(100-t)/100):
                outputimg[i,j]=0
            else :
                outputimg[i,j]=255
    return outputimg

In [10]:
def getRefLengths(img, offset=1):
    cols = img.shape[1]
    rows = img.shape[0]
    cur = 0
    hist = np.zeros((rows,rows), dtype=np.uint32)
    
    for i in range(0,cols):
        flag = False
        begin = 0
        blackRuns = []
        whiteRuns = []
        runtype = 0
        while begin < rows and img[begin,i] == 1:
            begin += 1
#         print('col=', i)
        for j in range(begin, rows):
            if img[j,i] == runtype:
                cur += 1
#                 print('runtype', runtype, 'cur',cur)
            else:
#                 print('else')
                if runtype == 0:
#                     print('append black', cur)
                    blackRuns.append(cur)
                else:
#                     print('append white', cur)
                    whiteRuns.append(cur)
                cur = 1
                runtype = img[j, i]
                if flag:
#                     print(blackRuns[-1], '+', whiteRuns[-1])
                    hist[blackRuns[-1], whiteRuns[-1]] += 1
                flag = True
#         if runtype == 0:
#             blackRuns.append(cur)
#         else:
#             whiteRuns.append(cur)
        cur = 0
#         if flag:
#             hist[blackRuns[-1], whiteRuns[-1]] += 1
        
    mx = np.max(hist)
    ind = np.where(hist == mx)
    return ind[0][0], ind[1][0]
  

In [8]:
paths = ['../../ex1.png', '../../ex2.png', '../../ex3.png', '../../ex4.png', '../../ex5.png']
for i in paths:
# i = '../../ex4.png'
    img = sk.img_as_ubyte(rgb2gray(io.imread(i)))
    thr = fr.threshold_otsu(img)
    binary = np.where(img>thr, 1, 0)
    staffHeight, spaceHeight = getRefLengths(binary)
    print('staff height =', staffHeight, 'space Height =', spaceHeight)

staff height = 2 space Height = 6
staff height = 2 space Height = 6
staff height = 1 space Height = 7
staff height = 2 space Height = 17
staff height = 1 space Height = 8
